# Emotion Detection in Twitter Data
The scope of this notebook is to implement a Naive-Bayes classifier that can be used to predict emotion in Twitter messages.

In [242]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re

Define the location of the training and testing sets.

In [243]:
# Global variables
dataDir = "Data/ssec-aggregated/"
trainFile = dataDir+"train-combined-0.0.csv"
testFile = dataDir+"test-combined-0.0.csv"


Here we define function to parse our training and testing sets.

**tokenizePhrase** takes a text message, removes @ mentions and # hashtags  as well as non-alphabetic characters and stop-words and returns the remaining words.

**parseSentence** Takes a line from the training or testing file,  separates and parses the sentiment fields and the message field and returns an array of boolean sentiment flags as well as a tokenised version of the sentence.

In [253]:
KnownSentiments = ["Anger", "Anticipation", "Disgust", "Fear", "Joy", "Sadness", "Surprise", "Trust"]
dropAts=True
dropHashtags=True

eng_stopwords = set(stopwords.words('english'))

def tokenizePhrase(phrase):
    # Drop @-references used in social media texts.
    if dropAts:
        phrase = re.sub("@[^ ]*", "", phrase)
        
    # Drop hashtags
    if dropHashtags:
        phrase = re.sub("#[^ ]*", "", phrase)
        
    # Change non-alphabetic characters to spaces
    phrase = re.sub("[^A-Za-z]", " ", phrase).lower()
    
    # Tokenize phrase while removing stop words and dropping tokens that are not more than 1 char long.
    tokens = [w for w in word_tokenize(phrase) if w not in eng_stopwords and len(w)>1]
    
    return tokens
    
def parseSentence(sent):
    # The sentiment labels are encoded at the beginning of the line as tab-separated fields
    # Split the line by tabs so as to extract the labels and the text
    parts = sent.split("\t")
    
    if len(parts)<9:
        return ([], [])
    
    sentSents = parts[:8]
    
    # Match the sentiment labels with the known sentiments to extract a boolean vector 
    # encoding which sentiments are present.
    sentMap = [sentSents[i]==KnownSentiments[i] for i in range(0, len(sentSents))]

    
    # The actual text
    phrase = parts[8]
    
    tokens = tokenizePhrase(phrase)
    
    #return the sentiment map and the tokens extracted from the phrase
    return (sentMap, tokens)
    



The **BoW** class implements Bag of Words. 

The **fit** method takes a collection of tokens and adds them to the Bag of Words.

The **transform** method takes a collection of tokens and returns a term-document vector correspondnding to the tokens

The **fit_transform** method combines the other fit and transform methods into one.

In [245]:
# Bag of Words
class BoW:

    def fit(self, phraseTokens):
        for tok in phraseTokens:
            if tok not in self.vocabulary_:
                tok_ndx = len(self.vocabulary_)
                self.vocabulary_[tok]=tok_ndx
                self.index_[tok_ndx]=tok
                 
    def transform(self, phraseTokens):
        return [i for i in [self.vocabulary_.get(t, None) for t in phraseTokens] if i is not None]
    
    def fit_transform(self, phraseTokens):
        self.fit(phraseTokens)
        return self.transform(phraseTokens)
    
    vocabulary_=dict()
    index_=dict()


Class **NaiveBayes1** is an implementation of the Naive Bayes multinomial classifier algorithm. 

Apart from label-wise prediction, NaiveBayes1 also maintains pairwise probabilities of labels, so for every pair of labels (X,Y), the class keeps track of P(X|Y), P(¬X|Y), P(X|¬Y) and P(¬X|¬Y). It also has the capability of accepting a set of pairwise dependencies it can use to alter the final prediction probabilities. We can therefore, for example, set the classifier to consider the joint probability P(Anger, Fear) in order to attempt to take advantage of correlations between Anger and Fear to improve the prediction scores of the classifier.

In [246]:
import math


class NaiveBayes1:
    def __init__(self, classes):

        self.classes_=list(classes)
        print "Initialising NaiveBayes1 class with ", len(self.classes_), " classes..."
        
        self.globalCounts = np.zeros(1000) # counts for each word token
        self.labelWordCounts = np.zeros((len(classes), 1000)) # counts for each word token per label
        
        # pairWiseCountsMatched is a matrix of pairwise label co-incidence counts
        # i.e. for every instance where label x and label y are both true, element [x,y] is incremented
        # Also for every instance where label x and label y are both false, element [y,x] is incremented
        # Hence one half of the matrix holds positive incidence of labels while the other half hold negative incidence.
        self.pairwiseCountsMatched = np.zeros((len(classes), len(classes))) 
        
        # pairwiseCountsMismatched is a matrix of pairwise label non-incidence counts
        # i.e. for every instance where label x is true and label y in false, element [x,y] is incremented
        # for every instance where label x is false and label y is true, element [y,x] is incremented
        self.pairwiseCountsMismatched = np.zeros((len(classes), len(classes)))
        
        # Holds pairwise dependencies between classes.
        # A True in element [x,y] instructs the classifier to compute P(x,y:X) during prediction.
        # This is used to hopefully leverage correlations between classes to improve predictions.
        self.pairwiseIndex = np.zeros((len(classes), len(classes)))
        
        # labelCounts holds the incidence count for every class.
        # i.e. for every training instance labelled with x, labelCounts[x] is incremented
        self.labelCounts = np.zeros(len(classes))
        
        # The number of training instances
        self.documentCount = 0
        
    # Update counts with a new training example
    def update(self, labels, tokens):
        self.documentCount += 1
        
        for x in range(0, len(self.classes_)):
            #print(x)
            if labels[x]:
                self.labelCounts[x] += 1
                
            for y in range(0, len(self.classes_)):
                if y>x:
                    if labels[x]:
                        if labels[y]:
                            self.pairwiseCountsMatched[x, y] += 1
                        else:
                            self.pairwiseCountsMismatched[x, y] += 1
                else:
                    if not labels[x]:
                        if not labels[y]:
                            self.pairwiseCountsMatched[x, y] += 1
                        else:
                            self.pairwiseCountsMismatched[x, y] += 1
                        
            
        for w in tokens:
            toAdd=0
            # Does the word exist in our vocab?
            if w >= len(self.globalCounts):
                # No - extend the counts array to accomodate
                toAdd = int(math.ceil((w-len(self.globalCounts)+1) / 1000.0) * 1000)
                self.globalCounts = np.append(self.globalCounts, np.zeros(toAdd))

                
            self.globalCounts[w] += 1

            if toAdd>0:
                self.labelWordCounts = np.append(self.labelWordCounts, np.zeros((len(self.classes_),toAdd)), axis=1)

            for x in range(0, len(self.classes_)):
                if labels[x]:
                    self.labelWordCounts[x, w] += 1                            

    # Recalculate all probabilities with the current counts
    def recalc(self):
        #print("self.documentCount", self.documentCount)
        #print("self.labelCounts=", self.labelCounts)
        globalTotal = sum(self.globalCounts)
        #print(globalTotal)
        
        classProbs = np.array(self.labelWordCounts)
        #print("classProbs", classProbs.shape)
        #print("sum(classProbs)", sum(sum(classProbs)))

        #print("self.labelWordCounts=", self.labelWordCounts+1)
        #print("self.globalCounts=", self.globalCounts+len(classProbs)+1)
        
        # Laplace smoothed word probability by emotion - p(w|e) 
        # P(w|e) = count(w when e)/count(w)
        self.wordClassPosProbs = (1.0 * classProbs+1)/(self.globalCounts+len(classProbs))# p(x|y)

        #print("self.wordClassPosProbs", self.wordClassPosProbs.shape)
        #print("self.wordClassPosProbs=", self.wordClassPosProbs)
        
        # emotion probability - p(e) 
        # p(e) = count()
        # sum classProbs per ROW to find total counts per class
        
        self.posClassProbs = (1.0 * self.labelCounts) / self.documentCount
        #print("self.posClassProbs", self.posClassProbs)
        #print("self.posClassProbs", self.posClassProbs.shape)
        
        classProbs = self.globalCounts - self.labelWordCounts#np.array(self.globalCounts) - classProbs
        #print("classProbs=", classProbs+1)
        
        # Laplace smoothed word probability by negative emotion - p(w|¬e)
        self.wordClassNegProbs = (1.0 * classProbs+1)/(self.globalCounts+len(classProbs)+1)

        #print("self.wordClassNegProbs", self.wordClassNegProbs.shape)
        #print("self.wordClassNegProbs=", self.wordClassNegProbs)
        
        # negative emotion probability - p(¬e) - element per class
        self.negClassProbs = (1.0 *(self.documentCount-self.labelCounts)) / self.documentCount
        #print("self.negClassProbs", self.negClassProbs)
        #print("self.negClassProbs", self.negClassProbs.shape)
        
        #print(self.posClassProbs + self.negClassProbs)
        
        # pairwise emotion conditional probability - p(e1 | e2) - class x class
        self.pairwiseClassProbs_x_y_matched = np.zeros((len(self.classes_), len(self.classes_)))
        for x in range(0, len(self.classes_)):
            for y in range(0, len(self.classes_)):
                if x==y:
                    self.pairwiseClassProbs_x_y_matched[x, y]=1
                else:
                    if y > x: # p(x|y)
                        self.pairwiseClassProbs_x_y_matched[x, y] = (1.0*self.pairwiseCountsMatched[x, y])/(sum(self.labelWordCounts[y]))
                        self.pairwiseClassProbs_x_y_matched[y, x] = (1.0*self.pairwiseCountsMatched[y, x])/(sum(self.globalCounts)-sum(self.labelWordCounts[y]))
                     
        # p(¬e1|e1), p(e1|¬e2)                
        self.pairwiseClassProbs_x_y_mismatched = np.zeros((len(self.classes_), len(self.classes_)))
        for x in range(0, len(self.classes_)):
            for y in range(0, len(self.classes_)):
                if x==y:
                    self.pairwiseClassProbs_x_y_mismatched[x, y]=0
                else:
                    if y > x:  
                        self.pairwiseClassProbs_x_y_mismatched[x, y] = (1.0*self.pairwiseCountsMismatched[x, y])/(sum(self.labelWordCounts[y])) # p(x|¬y)
                        self.pairwiseClassProbs_x_y_mismatched[y, x] = (1.0*self.pairwiseCountsMismatched[y, x])/(sum(self.globalCounts)-sum(self.labelWordCounts[y])) # p(¬x|y)

        # pairwise emotion conditional probability - p(e2 | e1) - class x class
        self.pairwiseClassProbs_y_x_matched = np.zeros((len(self.classes_), len(self.classes_)))
        for x in range(0, len(self.classes_)):
            for y in range(0, len(self.classes_)):
                if x==y:
                    self.pairwiseClassProbs_y_x_matched[x, y]=1
                else:
                    if y > x: # p(y|x)
                        self.pairwiseClassProbs_y_x_matched[x, y] = (1.0*self.pairwiseCountsMatched[x, y])/(sum(self.labelWordCounts[x]))
                        self.pairwiseClassProbs_y_x_matched[y, x] = (1.0*self.pairwiseCountsMatched[y, x])/(sum(self.globalCounts)-sum(self.labelWordCounts[x])) #p(¬y|¬x)
        
        # p(¬e2|e1), p(e2|¬e1)               
        self.pairwiseClassProbs_y_x_mismatched = np.zeros((len(self.classes_), len(self.classes_)))
        for x in range(0, len(self.classes_)):
            for y in range(0, len(self.classes_)):
                if x==y:
                    self.pairwiseClassProbs_y_x_mismatched[x, y]=0
                else:
                    if y > x: # p(y|¬x)
                        self.pairwiseClassProbs_y_x_mismatched[x, y] = (1.0*self.pairwiseCountsMismatched[x, y])/(sum(self.labelWordCounts[x]))
                        self.pairwiseClassProbs_y_x_mismatched[y, x] = (1.0*self.pairwiseCountsMismatched[y, x])/(sum(self.globalCounts)-sum(self.labelWordCounts[x])) #\p(¬y|x)
                        
    # set pairs of classes to correlate in results
    # classPairs : list(tuple(_0, _1))
    #
    # self.pairwiseIndex = matrix(len(classes), len(classes))
    def setPariwiseDependencies(self, classPairs):
        self.pairwiseIndex = np.zeros((len(self.classes_), len(self.classes_)))
        for pair in classPairs:
            self.pairwiseIndex[pair[0], pair[1]] = True
      
    # Return the trained conditional probability of classes X and Y
    # ------+--------------           
    #       \   x_pos
    # y_pos | true | false  
    # ------+------+-------
    # true  | x|y  | ¬x|y 
    # false | x|¬y | ¬x|¬y
    #-------+------+------
    def getConditionalProbability(self, X, Y, x_pos, y_pos):
        if Y > X:
            if x_pos:
                if not y_pos:
                    #print "#b", key, value
                    return self.pairwiseClassProbs_x_y_mismatched[X, Y]
                else:
                    return self.pairwiseClassProbs_x_y_matched[X, Y]

            else:
                if not y_pos:
                    return self.pairwiseClassProbs_x_y_matched[Y, X]
                else:
                    return self.pairwiseClassProbs_x_y_mismatched[Y, X]
        else:
            if x_pos:
                if not y_pos:
                    return self.pairwiseClassProbs_y_x_mismatched[Y, X]
                else:

                    return self.pairwiseClassProbs_y_x_matched[Y, X]
            else:
                if not y_pos:
                    return self.pairwiseClassProbs_y_x_matched[ X, Y] 
                else:
                    return self.pairwiseClassProbs_y_x_mismatched[X, Y] 

    
    # Classify a tokenised test instance
    def classify(self, tokens):
        # Predictions per class
        classPredictions = [False] * len(self.classes_)
        
        # class probabilities; positive and negative
        posClassProbs=[1.0]*len(classPredictions)
        negClassProbs=[1.0]*len(classPredictions)
        
        # Compute P(Class|Token) and P(¬Class|Token)
        for tok in tokens:
            if tok < len(self.globalCounts):
                posClassProbs *= self.wordClassPosProbs[:, tok]
                negClassProbs *= self.wordClassNegProbs[:, tok]
                
        posClassProbs *= self.posClassProbs
        negClassProbs *= self.negClassProbs
        
        preds = posClassProbs>negClassProbs

        # Calculate pairwise probabilities
        #p(X, Y | W) = p(X|Y).p(X).p(Y).Prod(w|X, w|Y)
        for x in range(0, len(self.classes_)):
            for y in range(0, len(self.classes_)):
                if self.pairwiseIndex[x, y]:
                    probs = np.zeros((2,2))

                    probs[1, 1] = self.getConditionalProbability(x, y, True, True)
                    probs[1, 0] = self.getConditionalProbability(x, y, True, False)
                    probs[0, 1] = self.getConditionalProbability(x, y, False, True)
                    probs[0, 0] = self.getConditionalProbability(x, y, False, False)

                    probs[1,:] *= posClassProbs[x]
                    probs[0,:] *= negClassProbs[x]
                    probs[:,1] *= posClassProbs[y]
                    probs[:,0] *= negClassProbs[y]

                    argmax = np.argmax(probs)
                    argmax_prob = (argmax/probs.shape[1], argmax % probs.shape[1])

                    preds[x] = bool(argmax_prob[0])
                    preds[y] = bool(argmax_prob[1])

        
        return preds
                    
        
    classes_=list()
    globalCounts=np.array([]) # array of counts indexed by token id
    labelWordCounts=np.array([]) # array of classes_ rows x tokens columns
    # array classes x classes x tokens
    # for y>x (top diagonal half) contains counts where sentiment(x) and sentiment(y)
    # for x<y (bottom diagonal half) contains counts where not(sentiment(x)) and not(sentiment(y))
    pairwiseCountsMatched=np.array([]) 
    
    # array classes x classes x tokens
    # for y>x (top diagonal half) contains counts where sentiment(x) and not(sentiment(y))
    # for x<y (bottom diagonal half) contains counts where not(sentiment(x)) and sentiment(y)
    pairwiseCountsMismatched=np.array([]) 
    classPairs=dict() # dict of emotion id -> emotion id
    pairwiseIndex=np.array([])

Class ModelScores calculates and maintains prediction scores for a classificiation model

In [301]:
class ModelScores:
    def __init__(self, knownClasses):
        self.knownClasses = knownClasses
        self.truePositives = np.zeros(len(knownClasses))
        self.trueNegatives = np.zeros(len(knownClasses))
        self.falsePositives = np.zeros(len(knownClasses))
        self.falseNegatives = np.zeros(len(knownClasses))
        
    def accumulate(self, predictedLabels, trainingLabels):
        self.truePositives += np.logical_and(predictedLabels, trainingLabels)
        self.trueNegatives += np.logical_and(np.logical_not(predictedLabels), np.logical_not(trainingLabels))
        self.falsePositives += np.logical_and(preds, np.logical_not(trainingLabels))
        self.falseNegatives += np.logical_and(np.logical_not(preds), trainingLabels)

    def getStats(self):
        precision = self.truePositives / (self.truePositives+self.falsePositives)
        recall = self.truePositives / (self.truePositives+self.falseNegatives)
        accuracy = (self.truePositives+self.trueNegatives)/(self.truePositives+self.trueNegatives+self.falsePositives+self.falseNegatives)
        
        f1 = 2* ((precision * recall)/(precision + recall))
        
        return (accuracy, precision, recall, f1)
    
    def printScores(self):
        (accuracy, precision, recall, f1score) = self.getStats()
        print self.knownClasses
        print "true positives = ", self.truePositives
        print "true negatives = ", self.trueNegatives
        print "false positives = ", self.falsePositives
        print "false negatives = ", self.falseNegatives
        print
        print "accuracy=", np.round(accuracy, 3)
        print "precision=", np.round(precision, 3)
        print "recall=", np.round(recall, 3)
        print "f1score=", np.round(f1score, 3)
      
    def printScoresAsTable(self):
        (accuracy, precision, recall, f1score) = self.getStats()
        return pd.DataFrame(data=[self.truePositives, self.trueNegatives, \
                                  self.falsePositives, self.falseNegatives, \
                                  np.round(accuracy,3), np.round(precision, 3), \
                                  np.round(recall,3), np.round(f1score,3)],
                    index=["True Positives", "True Negatives", \
                           "False Positives", "False Negatives", \
                           "Accuracy", "Precision", "Recall", "F1-Score"],
                    columns=self.knownClasses)

    def printScoresAsTableWithDeltas(self, other):
        (accuracy, precision, recall, f1score) = self.getStats()
        (da, dp, dr, df1) = self.getDeltas(other)
        
        addDelta = lambda a, d: np.array([str(x)+"("+str(y)+")" for(x, y) in zip(a, d)])
        data=[self.truePositives, self.trueNegatives, \
                                  self.falsePositives, self.falseNegatives, \
                                  addDelta(np.round(accuracy,3), np.round(da,3)), 
                                  addDelta(np.round(precision, 3), np.round(dp, 3)), \
                                  addDelta(np.round(recall,3), np.round(dr, 3)), \
                                  addDelta(np.round(f1score,3), np.round(df1, 3))]
        
        print data
        return pd.DataFrame(data=data,
                    index=["True Positives", "True Negatives", \
                           "False Positives", "False Negatives", \
                           "Accuracy", "Precision", "Recall", "F1-Score"],
                    columns=self.knownClasses)
    
    def getDeltas(self, other):
        (o_accuracy, o_precision, o_recall, o_f1) = other.getStats()
        (accuracy, precision, recall, f1) = self.getStats()
        
        return (accuracy-o_accuracy, precision-o_precision, recall-o_recall, f1-o_f1)
    
    def printDeltas(self, other):
        (accuracy, precision, recall, f1) = self.getDeltas(other)
        
        print "delta accuracy=", np.round(accuracy, 3)
        print "delta precision=", np.round(precision, 3)
        print "delta recall=", np.round(recall, 3)
        print "delta f1score=", np.round(f1, 3)
        

## Part 1: Individual Emotions

Predict individual emotions based on the training data. 

No dependencies between different emotions.

In [248]:
bow = BoW() # Create empty Bag of Words
nb1 = NaiveBayes1(KnownSentiments) # Create Naive Bayes class with the known sentiments as labels.
i=0
# Process the training file line-by-line
for line in open(trainFile):
    #print i, line
    (classMap, tokens)=parseSentence(line) # Parse each sentence returning sentiments and tokens
    #print(i, ": ", classMap)
    if len(classMap)==0:
        i+=1
        continue
        
    token_ndx = bow.fit_transform(tokens) # Transform tokens to term-document frequency
    
    nb1.update(classMap, token_ndx) # Update the Naive Bayes class with the new training data
    
    i+=1
    
nb1.recalc() # Recalculate the probabilities
print "Done. Processed ", i, " instances"

Initialising NaiveBayes1 class with  8  classes...
Done. Processed  2914  instances


In [249]:
nb1.setPariwiseDependencies({}) # Assume NO pairwise dependencies between labels
origScores = ModelScores(KnownSentiments)

# Process the test file
for line in open(testFile):
    # Parse, tokenise and BoW...
    (testClassMap, tokens) = parseSentence(line)
    if (tokens is None):
        continue
    tokens = bow.transform(tokens)
    # ... and use Naive Bayes to classify
    preds = nb1.classify(tokens)
     
    origScores.accumulate(preds, testClassMap)

origScores.printScores()
(orig_accuracy, orig_precision, orig_recall, orig_f1) = origScores.getStats()
origScores.printScoresAsTable()

['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']
true positives =  [1176.  977.  477.  105.  353.  956.    2.  115.]
true negatives =  [ 180.  250.  777. 1099.  938.  200. 1426. 1233.]
false positives =  [531. 501. 267.  57. 261. 695.   3.  42.]
false negatives =  [ 69. 228. 435. 695. 404. 105. 525. 566.]

accuracy= [0.693 0.627 0.641 0.616 0.66  0.591 0.73  0.689]
precision= [0.689 0.661 0.641 0.648 0.575 0.579 0.4   0.732]
recall= [0.945 0.811 0.523 0.131 0.466 0.901 0.004 0.169]
f1score= [0.797 0.728 0.576 0.218 0.515 0.705 0.008 0.274]


,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
True Positives,1176.000,977.000,477.000,105.000,353.000,956.000,2.000,115.000
True Negatives,180.000,250.000,777.000,1099.000,938.000,200.000,1426.000,1233.000
False Positives,531.000,501.000,267.000,57.000,261.000,695.000,3.000,42.000
False Negatives,69.000,228.000,435.000,695.000,404.000,105.000,525.000,566.000
Accuracy,0.693,0.627,0.641,0.616,0.660,0.591,0.730,0.689
Precision,0.689,0.661,0.641,0.648,0.575,0.579,0.400,0.732
Recall,0.945,0.811,0.523,0.131,0.466,0.901,0.004,0.169
F1-Score,0.797,0.728,0.576,0.218,0.515,0.705,0.008,0.274


As can be seen above, there are a couple of problem points in our predictions. Sadness and Joy have very low recalls, resulting in very low f1 scores. Accuracy is good all around, however accuracy alone as a performance indicator is misleading.

## Part 2: Manual addition of relationships between emotion labels

### Part 2a: Pariwise combinations of 2 emotion labels.

We notice from the above scores that while Sadness has the worst accuracy, Surprise, however has by far the worst f1-score, due to a very low recall. In fact, our classifier seems very reluctant to classify Sadness, having only scored 2 true positives on this label, compared to 525 false negatives.

We tried to improve the score on the Surprise label by adding a relationship to another emotion and we found that pairing Surprise to Anger gave the best, albeit modest, improvement.

In [271]:
# Anger | Sadness
indexes=[]
columns=["E1 f1-Score", "E1 f1-delta", "E2 f1-Score", "E2 f1-delta"]
f1scores=np.zeros(((len(KnownSentiments)-1)*2, 4))

def validate(e1, e2):
    indexes.extend([KnownSentiments[e1]+"->"+KnownSentiments[e2]])
    nb1.setPariwiseDependencies([(e1,e2)])
    scores = ModelScores(KnownSentiments)

    for line in open(testFile):
        (testClassMap, tokens) = parseSentence(line)
        if (tokens is None):
            continue
        tokens = bow.transform(tokens)
        preds = nb1.classify(tokens)

        scores.accumulate(preds, testClassMap)

    (a, r, p, f1) = scores.getStats()
    (da, dr, dp, df1) = scores.getDeltas(origScores)
    f1scores[i, 0] = f1[e1]
    f1scores[i, 1] = df1[e1]
    f1scores[i, 2] = f1[e2]
    f1scores[i, 3] = df1[e2]

i=0
for e in range(0, 8):
    if e!=6:
        validate(6, e)
        i+=1
        
for e in range(0,8):
    if e!=6:
        validate(e, 6)
        i+=1
        
        #scores.printScores()

        #scores.printDeltas(origScores)
print columns
f1scores_df = pd.DataFrame(data=f1scores, columns=columns, index=indexes)

f1scores_df

/home/etienne/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()


['E1 f1-Score', 'E1 f1-delta', 'E2 f1-Score', 'E2 f1-delta']


,E1 f1-Score,E1 f1-delta,E2 f1-Score,E2 f1-delta
Surprise->Anger,NaN,NaN,0.770719,-0.026029
Surprise->Anticipation,0.011257,0.003738,0.729174,0.000885
Surprise->Disgust,NaN,NaN,0.480638,-0.095449
Surprise->Fear,0.007533,0.000014,0.095462,-0.122834
Surprise->Joy,0.003781,-0.003738,0.470999,-0.043953
Surprise->Sadness,NaN,NaN,0.682737,-0.022278
Surprise->Trust,NaN,NaN,0.154037,-0.120426
Anger->Surprise,0.777046,-0.019702,NaN,NaN
Anticipation->Surprise,0.755772,0.027483,0.011257,0.003738
Disgust->Surprise,0.600000,0.023913,NaN,NaN


## Part 2b - Best set of pairwise variable assignments
We now want to search through all pairwise combinations of emotions and all four combinations of conditional probability assignments (x|y, x|¬y, ¬x|y, ¬x|¬y) to find the best assignments. 

In [204]:
# Save the original performance metrics for the unbiased naive bayes model so that we can
# determine which pairwise assignments resulted in improves predictive power.
(origAccuracy, origPrecision, origRecall, origf1) = origScores.getStats()
mean_orig_f1 = np.mean(origf1) # Mean f1-score for ubnbiased model

# Store the f1 score deltas for every pairwise combination
pairwiseImprovements = np.zeros((len(KnownSentiments), len(KnownSentiments)))


for x in range(0, len(KnownSentiments)):
    for y in range(0, len(KnownSentiments)):
        if x!=y:
            sentx = KnownSentiments[x]
            senty = KnownSentiments[y]                
                
            print "Evaluating P(", sentx, ",", senty, " | X)"
            
            # Set the dependency in the classifier
            nb1.setPariwiseDependencies([(x, y)])

            # Construct a ModelScores object to hold the new scores
            scores = ModelScores(KnownSentiments)

            # Process the test file, classifying each instance and accumulated
            # the result in our ModelScores object
            for line in open(testFile):
                (testClassMap, tokens) = parseSentence(line)
                if (tokens is None):
                    continue
                tokens = bow.transform(tokens)
                preds = nb1.classify(tokens)

                scores.accumulate(preds, testClassMap)

            # Retrieve the new scores...
            (a, p, r, f1) = scores.getStats()
            # ... and calculate how much they've changed from the unbiased model
            (d_a, d_p, d_r, d_f1) = scores.getDeltas(origScores)
            
            mean_f1 = np.mean(f1) # Mean f1-score of biased model
            mean_delta_f1 = mean_f1 - mean_orig_f1 # Change in f1-score
            print(mean_delta_f1, ":", np.mean(d_f1))
            #deltas[combination, y] = d_a[x]
            print "    Edge changed mean f1-score from ", mean_orig_f1, " to ", mean_f1, "(" , (mean_f1-mean_orig_f1), ")"
            pairwiseImprovements[x, y] = mean_delta_f1 # Save the delta
    

Evaluating P( Anger , Anticipation  | X)
(-0.001642360451014624, ':', -0.0016423604510146378)
    Edge changed mean f1-score from  0.4776710641922476  to  0.47602870374123296 ( -0.001642360451014624 )
Evaluating P( Anger , Disgust  | X)
(0.006635081319519287, ':', 0.006635081319519287)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4843061455117669 ( 0.006635081319519287 )
Evaluating P( Anger , Fear  | X)
(-0.0050065677835374744, ':', -0.005006567783537461)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4726644964087101 ( -0.0050065677835374744 )
Evaluating P( Anger , Joy  | X)
(-0.007342081390642297, ':', -0.0073420813906423035)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4703289828016053 ( -0.007342081390642297 )
Evaluating P( Anger , Sadness  | X)
(0.0009198558938562362, ':', 0.0009198558938562224)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4785909200861038 ( 0.0009198558938562362 )
Evaluating P( Anger , Surprise

/home/etienne/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()


(nan, ':', nan)
    Edge changed mean f1-score from  0.4776710641922476  to  nan ( nan )
Evaluating P( Anger , Trust  | X)
(-0.005444459350626429, ':', -0.005444459350626412)
    Edge changed mean f1-score from  0.4776710641922476  to  0.47222660484162116 ( -0.005444459350626429 )
Evaluating P( Anticipation , Anger  | X)
(-0.005093717398842568, ':', -0.005093717398842623)
    Edge changed mean f1-score from  0.4776710641922476  to  0.472577346793405 ( -0.005093717398842568 )
Evaluating P( Anticipation , Disgust  | X)
(-0.0057622878853967885, ':', -0.005762287885396858)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4719087763068508 ( -0.0057622878853967885 )
Evaluating P( Anticipation , Fear  | X)
(-0.0067718454252605476, ':', -0.006771845425260523)
    Edge changed mean f1-score from  0.4776710641922476  to  0.47089921876698704 ( -0.0067718454252605476 )
Evaluating P( Anticipation , Joy  | X)
(0.0014774749586726754, ':', 0.0014774749586727032)
    Edge changed mean f1-

/home/etienne/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in divide


(nan, ':', nan)
    Edge changed mean f1-score from  0.4776710641922476  to  nan ( nan )
Evaluating P( Surprise , Fear  | X)
(-0.010113478863478798, ':', -0.010113478863478867)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4675575853287688 ( -0.010113478863478798 )
Evaluating P( Surprise , Joy  | X)
(-0.0010597838473834775, ':', -0.0010597838473834706)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4766112803448641 ( -0.0010597838473834775 )
Evaluating P( Surprise , Sadness  | X)
(nan, ':', nan)
    Edge changed mean f1-score from  0.4776710641922476  to  nan ( nan )
Evaluating P( Surprise , Trust  | X)
(nan, ':', nan)
    Edge changed mean f1-score from  0.4776710641922476  to  nan ( nan )
Evaluating P( Trust , Anger  | X)
(-0.004941079993604491, ':', -0.004941079993604481)
    Edge changed mean f1-score from  0.4776710641922476  to  0.4727299841986431 ( -0.004941079993604491 )
Evaluating P( Trust , Anticipation  | X)
(-0.005751196035703676, ':', -0.00

Process the beneficial assignments discovered in the previous cell and configure a new classifier with the best ones that don't conflict.

i.e. if a beneficial relationship is Anger|Fear and we configure it into the  new classifier, we cannot configure the classifier with any other relationships involving Anger or Fear since we are only considering pairwise relationships.

In [279]:
np.round(pairwiseImprovements, 3)

array([[ 0.   , -0.002,  0.007, -0.005, -0.007,  0.001,    nan, -0.005],
       [-0.005,  0.   , -0.006, -0.007,  0.001, -0.002,  0.005, -0.001],
       [ 0.003, -0.008,  0.   , -0.006,  0.005,  0.004,    nan,  0.011],
       [-0.012, -0.009, -0.014,  0.   , -0.001,  0.006,    nan, -0.001],
       [-0.013, -0.002,  0.004,  0.   ,  0.   , -0.007,  0.007, -0.   ],
       [ 0.001, -0.   ,  0.006,  0.021, -0.003,  0.   ,    nan, -0.007],
       [   nan,  0.001,    nan, -0.01 , -0.001,    nan,  0.   ,    nan],
       [-0.005, -0.006,  0.01 , -0.005, -0.006, -0.008,    nan,  0.   ]])

In [205]:
# Find the assignments that gave the best improvements
sortedNdxs = np.argsort(pairwiseImprovements.flatten())[::-1]
usedLabels = set()
globalPariwiseAssignments = list()

for ndx in sortedNdxs:
    #print ndx
    x = ndx / len(KnownSentiments)
    y = ndx % len(KnownSentiments)
   # print(x, y)
    if pairwiseImprovements[x, y]>0:
        print "Beneficial Edge: ", KnownSentiments[x], "|", KnownSentiments[y], ": f1-score +", pairwiseImprovements[x, y]
        
        if (x not in usedLabels) and (y not in usedLabels):
            usedLabels.update([x])
            usedLabels.update([y])
            print(x, y)
            globalPariwiseAssignments.extend([(x, y)])
            
            

Beneficial Edge:  Sadness | Fear : f1-score + 0.02070378128161554
(5, 3)
Beneficial Edge:  Disgust | Trust : f1-score + 0.010669109815159339
(2, 7)
Beneficial Edge:  Trust | Disgust : f1-score + 0.009642641338028668
Beneficial Edge:  Joy | Surprise : f1-score + 0.0072825240220578125
(4, 6)
Beneficial Edge:  Anger | Disgust : f1-score + 0.006635081319519287
Beneficial Edge:  Sadness | Disgust : f1-score + 0.006092191324433505
Beneficial Edge:  Fear | Sadness : f1-score + 0.005806208443920657
Beneficial Edge:  Disgust | Joy : f1-score + 0.004879523852727385
Beneficial Edge:  Anticipation | Surprise : f1-score + 0.0046823278343668395
Beneficial Edge:  Joy | Disgust : f1-score + 0.0043465603982680845
Beneficial Edge:  Disgust | Sadness : f1-score + 0.00375586807822359
Beneficial Edge:  Disgust | Anger : f1-score + 0.003414307927609883
Beneficial Edge:  Anticipation | Joy : f1-score + 0.0014774749586726754
Beneficial Edge:  Surprise | Anticipation : f1-score + 0.0014432361233498447
Benefici

Now re-classify the test set using the discovered relations.

In [302]:
print globalPariwiseAssignments

nb1.setPariwiseDependencies(globalPariwiseAssignments)

scores = ModelScores(KnownSentiments)

for line in open(testFile):
    (testClassMap, tokens) = parseSentence(line)
    if (tokens is None):
        continue
    tokens = bow.transform(tokens)
    preds = nb1.classify(tokens)
     
    scores.accumulate(preds, testClassMap)

[(5, 3), (2, 7), (4, 6)]


In [304]:
scores.printScoresAsTableWithDeltas(origScores)

[array([1.176e+03, 9.770e+02, 6.320e+02, 2.550e+02, 5.380e+02, 7.980e+02,
       1.000e+00, 1.300e+02]), array([ 180.,  250.,  584.,  976.,  629.,  393., 1429., 1234.]), array([531., 501., 460., 180., 570., 502.,   0.,  41.]), array([ 69., 228., 280., 545., 219., 263., 526., 551.]), array(['0.693(0.0)', '0.627(0.0)', '0.622(-0.019)', '0.629(0.014)',
       '0.597(-0.063)', '0.609(0.018)', '0.731(0.001)', '0.697(0.008)'],
      dtype='|S13'), array(['0.689(0.0)', '0.661(0.0)', '0.579(-0.062)', '0.586(-0.062)',
       '0.486(-0.089)', '0.614(0.035)', '1.0(0.6)', '0.76(0.028)'],
      dtype='|S13'), array(['0.945(0.0)', '0.811(0.0)', '0.693(0.17)', '0.319(0.187)',
       '0.711(0.244)', '0.752(-0.149)', '0.002(-0.002)', '0.191(0.022)'],
      dtype='|S13'), array(['0.797(0.0)', '0.728(0.0)', '0.631(0.055)', '0.413(0.195)',
       '0.577(0.062)', '0.676(-0.029)', '0.004(-0.004)', '0.305(0.031)'],
      dtype='|S13')]


,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
True Positives,1176,977,632,255,538,798,1,130
True Negatives,180,250,584,976,629,393,1429,1234
False Positives,531,501,460,180,570,502,0,41
False Negatives,69,228,280,545,219,263,526,551
Accuracy,0.693(0.0),0.627(0.0),0.622(-0.019),0.629(0.014),0.597(-0.063),0.609(0.018),0.731(0.001),0.697(0.008)
Precision,0.689(0.0),0.661(0.0),0.579(-0.062),0.586(-0.062),0.486(-0.089),0.614(0.035),1.0(0.6),0.76(0.028)
Recall,0.945(0.0),0.811(0.0),0.693(0.17),0.319(0.187),0.711(0.244),0.752(-0.149),0.002(-0.002),0.191(0.022)
F1-Score,0.797(0.0),0.728(0.0),0.631(0.055),0.413(0.195),0.577(0.062),0.676(-0.029),0.004(-0.004),0.305(0.031)


In [305]:
scores.printDeltas(origScores)

(a, p, r, f1) = scores.getStats()

print
print "Change in f1-score: ", np.round(mean_orig_f1,3), "->", np.round(np.mean(f1),2), " = ", np.round((np.mean(f1)-mean_orig_f1), 3)

delta accuracy= [ 0.     0.    -0.019  0.014 -0.063  0.018  0.001  0.008]
delta precision= [ 0.     0.    -0.062 -0.062 -0.089  0.035  0.6    0.028]
delta recall= [ 0.     0.     0.17   0.187  0.244 -0.149 -0.002  0.022]
delta f1score= [ 0.     0.     0.055  0.195  0.062 -0.029 -0.004  0.031]

Change in f1-score:  0.478 -> 0.52  =  0.039


So from the above results it appears that we managed to increase the scores in all classes except anticipation which dropped .5% in accuracy. Clearly this strategy, while, on the whole, increasing the predictive performance of th model is not optimal. A perfect method would be to go through th entire search space of the problem, but for pairwise matching of emotions that would mean (8x8x4)! possible configurations, but this is not tractable.

A possible better method to search through this state space would possibly be a genetic algorithm.

In [2]:
import numpy as np

t = np.zeros((2, 3))
t[0,0]=1
t[1,0]=2
t[0,1]=2
t[1,2]=5
print(t)

#print(len(t))
#t2 = np.sum(t, axis=1)
#print(t2)
#t2/sum(t2)

[[1. 2. 0.]
 [2. 0. 5.]]


In [3]:
sum(t)

array([3., 2., 5.])

In [4]:
len(t)

2

In [5]:
t2 = [2, 2, 3]

t/t2

array([[0.5       , 1.        , 0.        ],
       [1.        , 0.        , 1.66666667]])

In [6]:
sum(t)-t

array([[2., 0., 5.],
       [1., 2., 0.]])

In [14]:
t
t[0,1] = 10
t

array([[ 1., 10.,  0.],
       [ 2.,  0.,  5.]])

In [15]:
print(np.argmax(t)/t.shape[1], np.argmax(t) % t.shape[1])

(0, 1)
